In [1]:
import csv
import pandas as pd
import numpy as np
import re
from modelscope import AutoModelForCausalLM, AutoTokenizer, snapshot_download
from modelscope import GenerationConfig

2024-07-03 19:17:00,626 - modelscope - INFO - PyTorch version 2.3.1 Found.
2024-07-03 19:17:00,627 - modelscope - INFO - Loading ast index from /mnt/workspace/.cache/modelscope/ast_indexer
2024-07-03 19:17:00,653 - modelscope - INFO - Loading done! Current index file version is 1.15.0, with md5 3b674e250eaff3ed305bb955cabf2c95 and a total number of 980 components indexed
/root/miniconda3/envs/qwen/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# TDOO 原版用14B-Chat，太大加载不了，只能换Int4版本
model_dir = '/mnt/workspace/Tongyi-Finance-14B-Chat-Int4'

In [3]:
# Note: The default behavior now has injection attack prevention off.
tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)

In [4]:
new_question_file_dir = '/mnt/workspace/intermediate/question_csv.csv'
new_question_file = pd.read_csv(new_question_file_dir,delimiter = ",",header = 0)

In [5]:
company_file_dir = '/mnt/workspace/files/AF0_pdf_to_company.csv'
company_file = pd.read_csv(company_file_dir,delimiter = ",",header = 0)
company_list = list()
for cyc in range(len(company_file)):
    company_list.append(company_file[cyc:cyc+1]['公司名称'][cyc])

In [6]:
model = AutoModelForCausalLM.from_pretrained(model_dir, device_map="cuda:0", trust_remote_code=True, 
                                             # bf16=True
                                            ).eval()
model.generation_config = GenerationConfig.from_pretrained(model_dir,
                                                           trust_remote_code=True,
                                                           temperature = 0.0000001,
                                                           top_p = 1,
                                                           do_sample = False,
                                                           seed = 1234)

print('A01_model_loaded')

CUDA extension not installed.


A01_model_loaded


/root/miniconda3/envs/qwen/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1e-07` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/root/miniconda3/envs/qwen/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:407: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [7]:
g = open('/mnt/workspace/intermediate/A01_question_classify.csv', 'w', newline='', encoding = 'utf-8-sig') 
csvwriter = csv.writer(g)
csvwriter.writerow(['问题id','问题','答案','分类'])

15

In [8]:
prompt = """
    你是一个问题分类器。对于每个提供给你的问题，你需要猜测答案是在该公司的招股说明书中还是在基金股票数据库里。以下是一些例子：

    问题：“在2019年的中期报告里，XX基金管理有限公司管理的基金中，有多少比例的基金是个人投资者持有的份额超过机构投资者？希望得到一个精确到两位小数的百分比。”
    回答：“基金股票数据库”
    
    问题：“XXXX股份有限公司变更设立时作为发起人的法人有哪些？”
    回答：“该公司的招股说明书”
    
    问题：“我想知道XXXXXX债券A基金在20200930的季报中，其可转债持仓占比最大的是哪个行业？用申万一级行业来统计。”
    回答：“基金股票数据库”
    
    问题：“XXXXXX股份有限公司2020年增资后的投后估值是多少？”
    回答：“该公司的招股说明书”
    
    问题：“根据XXXXXX股份有限公司招股意向书，全球率先整体用LED路灯替换传统路灯的案例是？”
    回答：“该公司的招股说明书”
    
    问题：“什么公司、在何时与XXXXXX股份有限公司发生了产品争议事项？产品争议事项是否已经解决？”
    回答：“该公司的招股说明书”
    
    问题：“请帮我查询下股票代码为XXXXXX的股票在2021年内最高日收盘价是多少？”
    回答：“基金股票数据库”
    
    问题：“XXXXXX股份有限公司的中标里程覆盖率为多少？”
    回答：“该公司的招股说明书”
    
    问题：“根据中国证监会颁布的《上市公司行业分类指导》的规定，XXXXXX有限公司所属的行业大类、中类、小类是什么？”
    回答：“该公司的招股说明书”
    
    问题：“请问XXXX年一季度有多少家基金是净申购?它们的净申购份额加起来是多少?请四舍五入保留小数点两位。”
    回答：“基金股票数据库”
    
    问题：“XXXXXX有限公司和合肥翰林是否按规定为员工缴纳了社会保险？”
    回答：“该公司的招股说明书”
    
    问题：“我想知道XXXXXX有限公司在2020年成立了多少只管理费率小于0.8%的基金？”
    回答：“基金股票数据库”
    
    问题：“根据《CRCC产品认证实施规则》，《铁路产品认证证书》有效期为多久？XXXXXX有限公司取得 《铁路产品认证证书》后，至少多久需要接受一次监督？”
    回答：“该公司的招股说明书”
    
    问题：“我想知道XXXXXX基金管理有限公司在2019年成立了多少只管理费率小于0.8%的基金？”
    回答：“基金股票数据库”
    
    问题：“请问XXXX年一季度有多少家基金是净申购?它们的净申购份额加起来是多少?请四舍五入保留小数点两位。”
    回答：“基金股票数据库”
    
    问题：“我想知道XXXXXX有限公司在2019年成立了多少只管理费率小于0.8%的基金？”
    回答：“基金股票数据库”
    
    问题：“我想知道股票XXXXXX在申万行业分类下的二级行业是什么？用最新的数据。”
    回答：“基金股票数据库”
    
    问题：“请帮我查询下股票代码为XXXXXX的股票在2019年内最高日收盘价是多少？”
    回答：“基金股票数据库”
    
    问题：“股票XXXXXX在20200227日期中的收盘价是多少?（小数点保留3位）”
    回答：“基金股票数据库”
    
    问题：“截至2009年底，中海达、南方测绘合计占有国产品牌销售额的多大比例？”
    回答：“该公司的招股说明书”
    
    问题：“截止2005年12月31日，南岭化工厂的总资产和净资产分别是多少？”
    回答：“该公司的招股说明书”
    
    问题：“股票XXXXXX在20200227日期中的收盘价是多少?（小数点保留3位）”
    回答：“基金股票数据库”

    根据上面提供的例子对以下问题进行分类。
    问题：“
    """

In [9]:
from tqdm import tqdm

for cyc in tqdm(range(len(new_question_file))):
    temp_question = new_question_file[cyc:cyc+1]['问题'][cyc]

    prompt1 = prompt + temp_question + """？"""

    response_new, history_new = model.chat(tokenizer, prompt1, history=None)
    if cyc % 100 == 0:
        print(str(new_question_file[cyc:(cyc+1)]['问题id'][cyc]))

    if '招股说明书' in response_new and '股票数据库' not in response_new:
        temp_class = 'Text'
    elif '招股说明书' not in response_new and '股票数据库' in response_new:
        temp_class = 'SQL'
        for company_name in company_list:
            if company_name in temp_question:
                temp_class = 'Text'
    else:
        temp_class = 'SQL'
        for company_name in company_list:
            if company_name in temp_question:
                temp_class = 'Text'
    if cyc in [166,174]:
        temp_calss = 'Text'


    csvwriter.writerow([str(new_question_file[cyc:(cyc+1)]['问题id'][cyc]),
                    str(new_question_file[cyc:(cyc+1)]['问题'][cyc]),
                    response_new,temp_class])

  0%|          | 0/1000 [00:00<?, ?it/s]/root/miniconda3/envs/qwen/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1e-07` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/root/miniconda3/envs/qwen/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:407: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
  0%|          | 1/1000 [00:08<2:26:45,  8.81s/it]

0


 10%|█         | 101/1000 [17:09<1:44:55,  7.00s/it]

100


 20%|██        | 201/1000 [32:47<2:43:25, 12.27s/it]

...

 80%|████████  | 801/1000 [2:10:40<36:12, 10.92s/it]  

800


 90%|█████████ | 901/1000 [2:28:07<10:49,  6.56s/it]  

900


100%|██████████| 1000/1000 [2:45:07<00:00,  9.91s/it]


In [10]:
g.close()